# Get Flood Data from National Flood Center


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
# import time
# from datetime import datetime
# from scipy.stats import linregress
# import math



In [2]:
#Read Data from HCAD system
df = pd.read_csv("Resources/Inner_Loop_18_19.csv")
df.info()

C:\Users\adria\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047078 entries, 0 to 1047077
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1047078 non-null  int64  
 1   ACCOUNT                1047078 non-null  int64  
 2   TAX_YEAR               1047078 non-null  int64  
 3   SITE_ADDR_1            1047078 non-null  object 
 4   SITE_ADDR_2            1047078 non-null  object 
 5   SITE_ADDR_3            1047078 non-null  object 
 6   STATE_CLASS            1047078 non-null  object 
 7   SCHOOL_DIST            1047078 non-null  object 
 8   NEIGHBORHOOD_CODE      1047078 non-null  float64
 9   NEIGHBORHOOD_GROUP     1047078 non-null  int64  
 10  MARKET_AREA_1_DSCR     1047078 non-null  object 
 11  MARKET_AREA_2_DSCR     1047078 non-null  object 
 12  TOTAL_BUILDING_AREA    1047078 non-null  int64  
 13  TOTAL_LAND_AREA        1047078 non-null  int64  
 14  ACREAGE           

In [3]:
#Concatenate Address for use in flood data
convert_dict={'SITE_ADDR_3':str}
df=df.astype(convert_dict)
df['Full Address']=df['SITE_ADDR_1'] +', '+'Houston,'+' Texas '+ df['SITE_ADDR_3']
df.head()

,Unnamed: 0,ACCOUNT,TAX_YEAR,SITE_ADDR_1,SITE_ADDR_2,SITE_ADDR_3,STATE_CLASS,SCHOOL_DIST,NEIGHBORHOOD_CODE,NEIGHBORHOOD_GROUP,...,TOTAL_BUILDING_AREA,TOTAL_LAND_AREA,ACREAGE,LAND_VALUE,ASSESSED_VALUE,TOTAL_APPRAISED_VALUE,TOTAL_MARKET_VALUE,LAST_INSPECTED_DATE,NEW_OWNER_DATE,Full Address
0,1085,21440000001,2019,2120 LIVE OAK ST,HOUSTON,77003,A1,01,8400.12,1301,...,2537,5000,0.1148,125000.0,360475.0,193261.0,360475.0,2016-05-11 11:46:44.107,2012-09-11 00:00:00.000,"2120 LIVE OAK ST HOUSTON, Texas 77003"
1,1088,21440000008,2019,2113 SAINT CHARLES ST,HOUSTON,77003,A1,01,8400.12,1301,...,1312,5000,0.1148,125000.0,125100.0,125100.0,125100.0,2013-12-17 17:05:42.620,2017-09-20 00:00:00.000,"2113 SAINT CHARLES ST HOUSTON, Texas 77003"
2,1125,21480000002,2019,2116 HUTCHINS ST,HOUSTON,77003,A1,01,8400.12,1301,...,1496,5000,0.1148,125000.0,128297.0,114370.0,128297.0,2013-12-16 12:13:21.600,2004-07-28 00:00:00.000,"2116 HUTCHINS ST HOUSTON, Texas 77003"
3,1170,21650000007,2019,2217 HUTCHINS ST,HOUSTON,77003,A1,01,8400.12,1301,...,3387,5000,0.1148,125000.0,527887.0,527887.0,527887.0,2015-12-02 14:38:09.433,2013-10-14 00:00:00.000,"2217 HUTCHINS ST HOUSTON, Texas 77003"
4,1171,21650000011,2019,2212 WEBSTER ST,HOUSTON,77003,A1,01,8400.12,1301,...,1508,6250,0.1435,140625.0,254430.0,241402.0,254430.0,2013-12-13 11:00:19.397,2001-05-05 00:00:00.000,"2212 WEBSTER ST HOUSTON, Texas 77003"


In [ ]:
# zips=df['SITE_ADDR_3'].unique().tolist()
ZIP=df['zip_code'].unique().tolist()
ZIP

In [7]:
# THE FOLLOWING CODE PULLS FLOOD DATA FOR EVERY ADDRESS FROM A LIST OF ZIP CODES
# The output are csv files per ZIP codes with latitude, longitude, flood description and flood zone 

# Import API key for flood data
from api_keys import flood_data_api_key
headers = {'x-api-key':flood_data_api_key}


#Call the API to National Flood Center per address and store data per ZIP Code
count=0
total_count=0

for zipcodes in ZIP:
    addresses=df.loc[df['SITE_ADDR_3']==zipcodes,'Full Address']
    available_addresses=[]
    lat=[]
    lng=[]
    flood_title=[]
    flood_zone=[]
    full_flood_data_df=pd.DataFrame()
    flood_data_df_temp=pd.DataFrame()
    full_flood_data_df_temp=pd.DataFrame()
    
# Temporary files are created every 10 calls to avoid loosing data if the connection fails
    for address in addresses:
         payload = {
        'address':address,
        'searchtype':'addressparcel',
        'getloma':'False'
            }
         flood_request = requests.get('https://api.nationalflooddata.com/data',headers=headers,params=payload).json()
         try:      
                flood_title.append(flood_request["result"]["flood.s_fld_haz_ar"][0]["zone_subty"])
                flood_zone.append(flood_request["result"]["flood.s_fld_haz_ar"][0]["fld_zone"])
                available_addresses.append(address)
                lat.append(flood_request["geocode"]['lat'])
                lng.append(flood_request["geocode"]['lng'])
                count=count+1
                total_count=total_count+1
                print (f'Processed Address#: {total_count} in Zip Code {zipcodes}')
                if count==10:
                    flood_dict_temp = {
                    "Address": available_addresses,
                    "Latitude": lat,
                    "Longitude": lng,
                    "Flood Description": flood_title,
                    "Flood Zone":flood_zone,
                    }
    
                    flood_data_df_temp = pd.DataFrame(flood_dict_temp)
                    flood_data_df_temp.to_csv(f'flood_data_{zipcodes}_temp.csv')
                    count=0
                    
         except:
                print (f'Skipping Address------------')
                continue
        
         flood_dict = {"Address": available_addresses,
                        "Latitude": lat,
                        "Longitude": lng,
                        "Flood Description": flood_title,
                        "Flood Zone":flood_zone,
                        }


         flood_data_df = pd.DataFrame(flood_dict)
         file=f'flood_data_{zipcodes}.csv'
         flood_data_df.to_csv(f'Output/flood_data_{zipcodes}.csv')     
         
        
flood_data_df.head()

Processed Address#: 1 in Zip Code 77284
Processed Address#: 2 in Zip Code 77284
Processed Address#: 3 in Zip Code 77284
Processed Address#: 4 in Zip Code 77284
Processed Address#: 5 in Zip Code 77284
Skipping Address------------
Processed Address#: 6 in Zip Code 77535
Skipping Address------------
Processed Address#: 7 in Zip Code 77535
Skipping Address------------
Skipping Address------------
Skipping Address------------
Skipping Address------------


,Address,Latitude,Longitude,Flood Description,Flood Zone
0,"22445 RAMSEY RD HUFFMAN, Texas 77535",30.019049,-95.050991,None,AE
1,"1574 WOLFE ISLAND RD DAYTON, Texas 77535",30.016879,-94.954048,FLOODWAY,AE
